In [1]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras_squeezenet import SqueezeNet
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical 
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import warnings
import argparse
import pickle
import cv2
import os
import sys
import glob
import matplotlib
matplotlib.use("Agg")

In [2]:
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

In [3]:
args = {
    "dataset": "./SCVD/5x3",
    "model": "./Squeezenet5x3/models/squeezenet_5x3.h5",
    "label-bin": "./Squeezenet5x3/label-bin/squeezenet_5x3.pickle",
    "epochs": 30,
    "plot": "./Squeezenet5x3/plots/squeezenet_5x3.png"
}

In [4]:
# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = set(["Violence", "NonViolence", "WeaponViolence"])

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print('-'*100)
print("[INFO] loading images...")
print('-'*100)
from imutils import paths
imagePaths = list(paths.list_images(args["dataset"]))
print(imagePaths)
data = []
labels = []

----------------------------------------------------------------------------------------------------
[INFO] loading images...
----------------------------------------------------------------------------------------------------
['./SCVD/5x3\\NonViolence\\nv100_merge_0.jpg', './SCVD/5x3\\NonViolence\\nv100_merge_1.jpg', './SCVD/5x3\\NonViolence\\nv100_merge_2.jpg', './SCVD/5x3\\NonViolence\\nv100_merge_3.jpg', './SCVD/5x3\\NonViolence\\nv100_merge_4.jpg', './SCVD/5x3\\NonViolence\\nv100_merge_5.jpg', './SCVD/5x3\\NonViolence\\nv100_merge_6.jpg', './SCVD/5x3\\NonViolence\\nv100_merge_7.jpg', './SCVD/5x3\\NonViolence\\nv100_merge_8.jpg', './SCVD/5x3\\NonViolence\\nv100_merge_9.jpg', './SCVD/5x3\\NonViolence\\nv101_merge_0.jpg', './SCVD/5x3\\NonViolence\\nv101_merge_1.jpg', './SCVD/5x3\\NonViolence\\nv101_merge_2.jpg', './SCVD/5x3\\NonViolence\\nv101_merge_3.jpg', './SCVD/5x3\\NonViolence\\nv101_merge_4.jpg', './SCVD/5x3\\NonViolence\\nv101_merge_5.jpg', './SCVD/5x3\\NonViolence\\nv101_merg

In [5]:
# loop over the image paths
for imagePath in tqdm(imagePaths[::]):
    # imagePath : file name ex) V_123
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2] # Violence / NonViolence

    # if the label of the current image is not part of of the labels
    # are interested in, then ignore the image
    if label not in LABELS:
        continue

    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

100%|██████████████████████████████████████████████████████████████████████████████| 5653/5653 [54:07<00:00,  1.74it/s]


In [6]:
# convert the data and labels to NumPy arrays
dt = np.array(data)
lbls = np.array(labels)


# perform one-hot encoding on the labels
lb = LabelBinarizer()
lbls_fit = lb.fit_transform(lbls)
#lbls_cat = to_categorical(lbls_fit, num_classes=3)

# partition the data into training and testing splits using 70% of
# the data for training and the remaining 30% for testing
(trainX, testX, trainY, testY) = train_test_split(dt, lbls_fit, test_size=0.2, stratify=lbls_fit, random_state=33)

In [7]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    #rotation_range=30,
    zoom_range=0.15,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")


# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

In [8]:
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [9]:
# load the InceptionV3 network, ensuring the head FC layer sets are left
# off
model = SqueezeNet(
    input_shape=None,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=3)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
model.trainable = True

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print('-'*100)
print("[INFO] compiling model...")
print('-'*100)
opt = SGD(lr=1e-3, momentum=0.9, decay=1e-4 / args["epochs"])
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print(model.summary())

----------------------------------------------------------------------------------------------------
[INFO] compiling model...
----------------------------------------------------------------------------------------------------
Model: "squeezenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 227, 227, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 113, 113, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                           

C:\Python310\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


                                                                                                  
 fire4/expand3x3 (Conv2D)       (None, 27, 27, 128)  36992       ['fire4/relu_squeeze1x1[0][0]']  
                                                                                                  
 fire4/relu_expand1x1 (Activati  (None, 27, 27, 128)  0          ['fire4/expand1x1[0][0]']        
 on)                                                                                              
                                                                                                  
 fire4/relu_expand3x3 (Activati  (None, 27, 27, 128)  0          ['fire4/expand3x3[0][0]']        
 on)                                                                                              
                                                                                                  
 fire4/concat (Concatenate)     (None, 27, 27, 256)  0           ['fire4/relu_expand1x1[0][0]',   
          

 fire8/concat (Concatenate)     (None, 13, 13, 512)  0           ['fire8/relu_expand1x1[0][0]',   
                                                                  'fire8/relu_expand3x3[0][0]']   
                                                                                                  
 fire9/squeeze1x1 (Conv2D)      (None, 13, 13, 64)   32832       ['fire8/concat[0][0]']           
                                                                                                  
 fire9/relu_squeeze1x1 (Activat  (None, 13, 13, 64)  0           ['fire9/squeeze1x1[0][0]']       
 ion)                                                                                             
                                                                                                  
 fire9/expand1x1 (Conv2D)       (None, 13, 13, 256)  16640       ['fire9/relu_squeeze1x1[0][0]']  
                                                                                                  
 fire9/exp

In [10]:
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print('-'*100)
print("[INFO] training head...")
print('-'*100)
H = model.fit(trainAug.flow(trainX, trainY, batch_size=32),
            steps_per_epoch=len(trainX) // 32,
            validation_data=valAug.flow(testX, testY),
            validation_steps=len(testX) // 32,
            epochs=args["epochs"])

----------------------------------------------------------------------------------------------------
[INFO] training head...
----------------------------------------------------------------------------------------------------
Epoch 1/30
141/141 [==============================] - 379s 3s/step - loss: 1.0982 - accuracy: 0.3452 - val_loss: 1.0971 - val_accuracy: 0.3643
Epoch 2/30
141/141 [==============================] - 350s 2s/step - loss: 1.0967 - accuracy: 0.3653 - val_loss: 1.0962 - val_accuracy: 0.3652
Epoch 3/30
141/141 [==============================] - 348s 2s/step - loss: 1.0961 - accuracy: 0.3657 - val_loss: 1.0955 - val_accuracy: 0.3661
Epoch 4/30
141/141 [==============================] - 349s 2s/step - loss: 1.0956 - accuracy: 0.3666 - val_loss: 1.0947 - val_accuracy: 0.3643
Epoch 5/30
141/141 [==============================] - 344s 2s/step - loss: 1.0954 - accuracy: 0.3646 - val_loss: 1.0927 - val_accuracy: 0.3652
Epoch 6/30
141/141 [==============================] - 346s 

In [11]:
# evaluate the network
print('-'*100)
print("[INFO] evaluating network...")
print('-'*100)
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

----------------------------------------------------------------------------------------------------
[INFO] evaluating network...
----------------------------------------------------------------------------------------------------


ValueError: in user code:

    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Python310\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Python310\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "squeezenet" is incompatible with the layer: expected shap

In [ ]:
# plot the training loss and accuracy
print('-'*100)
print("[INFO] plot the training loss and accuracy...")
print('-'*100)
N = args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

In [ ]:
# serialize the model to disk
print('-'*100)
print("[INFO] serializing network...")
print('-'*100)
model.save(args["model"])

In [ ]:
# serialize the label binarizer to disk
f = open(args["label-bin"], "wb")
f.write(pickle.dumps(lb))
f.close()